# Flight Data Processing and Analysis
### This Python script processes and analyzes flight data, including departure and arrival information merged with historical weather data. It categorizes flights by carrier, date, and time of day, providing insights into flight patterns and weather conditions. The script filters and cleans the data, facilitating further analysis and visualization.

#### Data Preprocessing:
#### Loads a CSV file into a DataFrame.
#### Maps carrier codes to main carrier classes.
#### Categorizes dates into Midweek, StartOfWeek, and Weekend.
#### Categorizes times into different time periods.
#### Replaces subcategories with main categories for weather phrases.
#### Extracts the year from the date.
#### Handles datetime objects, including custom time differences.
#### Removes unnecessary columns.
#### Data Analysis:
#### Performs analysis on specific columns to check for unique values.
#### Output:
#### Saves the cleaned DataFrame to a new CSV file.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
# Load the CSV file
df = pd.read_csv('arrival_depature_weather_merged_df.csv')

# print(df.head(1))

In [2]:
# keep: 
# Date (MM/DD/YYYY)_a

# categorical : Carrier Code_a, Origin Airport_a,clds_d,uv_desc_d,wdir_cardinal_d,wx_phrase_d,day_ind_d,
# nUMERICAL: uv_index_d,feels_like_d,snow_hrly_d,wspd_d	,precip_total_d	,precip_hrly_d,vis_d,	wc_d,dewPt_d	,heat_index_d,	rh_d,	pressure_d, 
# temp_d,Taxi-Out time (Minutes)_d	Delay Carrier (Minutes)_d	Delay Weather (Minutes)_d	Delay National Aviation System (Minutes)_d	Delay Security (Minutes)_d	Delay Late Aircraft Arrival (Minutes)_d
# Scheduled elapsed time (Minutes)_d	Actual elapsed time (Minutes)_d	Departure delay (Minutes)_d
# change  Scheduled departure time_d	Actual departure time_d

In [3]:
# drop:  Tail Number_a,Tail Number_d,Flight Number_a,valid_time_local_d,est_date_d,est_time_d,wx_icon_d,	icon_extd_d
# Origin Airport_d	DateTime_d	Unnamed: 0_w_d	expire_time_gmt_d	obs_id_d	valid_time_gmt_d
# Wheels-off time_d Destination Airport_d
# est_time_a	est_date_a	valid_time_local_a	Unnamed: 0_d_d	index_d	Carrier Code_d	Date (MM/DD/YYYY)_d	Flight Number_d	Tail Number_d	Destination Airport_d


In [4]:
# Define the mapping of subclasses to main classes
carrier_mapping = {
    'DL': 'DL', '9E': 'DL', 'OO': 'DL', 'EV': 'DL',
    'AA': 'AA', 'YX': 'AA', 'MQ': 'AA',
    'B6': 'B6', 'F9': 'B6',
    'UA': 'UA',
    'WN': 'WN'
}

# Map the Carrier Code_a to the main class
df['Main_Carrier'] = df['Carrier Code_a'].map(carrier_mapping)

In [5]:
# Convert the 'Date (MM/DD/YYYY)_a' column to datetime
df['Date (MM/DD/YYYY)_a'] = pd.to_datetime(df['Date (MM/DD/YYYY)_a'])


# Function to categorize the date into Midweek, StartOfWeek, and Weekend
def categorize_day(date):
    day_of_week = date.dayofweek  # Monday=0, Sunday=6
    if day_of_week == 0:
        return 'StartOfWeek'
    elif 1 <= day_of_week <= 3:
        return 'Midweek'
    else:
        return 'Weekend'

# Apply the function to create a new column
df['Week_Category'] = df['Date (MM/DD/YYYY)_a'].apply(categorize_day)

In [6]:
# Convert the Scheduled Arrival Time to a datetime object to extract the hour
df['Scheduled Arrival Time_a'] = pd.to_datetime(df['Scheduled Arrival Time_a'], format='%H:%M').dt.time

# Define the function to categorize the time
def categorize_time(time):
    hour = time.hour
    if 23 <= hour or hour < 4:
        return 'Night'
    elif 4 <= hour < 9:
        return 'Morning'
    elif 9 <= hour < 12:
        return 'Morning Peak'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 18:
        return 'Evening Peak'
    elif 18 <= hour < 23:
        return 'Evening'

# Apply the function to create a new column with the time categories
df['A_Time_of_Day'] = df['Scheduled Arrival Time_a'].apply(categorize_time)

In [7]:
# Replace 'Very High' with 'High' in 'uv_desc_a' and 'uv_desc_d'
df['uv_desc_a'] = df['uv_desc_a'].replace('Very High', 'High')
df['uv_desc_d'] = df['uv_desc_d'].replace('Very High', 'High')
df['uv_desc_d'] = df['uv_desc_d'].replace('Extreme', 'High')

In [8]:
# Complete mapping of subcategories to main categories for both columns
category_mapping = {
    # Cloudy category
    'Mostly Cloudy': 'Cloudy',
    'Unknown Precipitation': 'Cloudy',
    
    # Windy category
    'Blowing Snow / Windy': 'Windy',
    'Cloudy / Windy': 'Windy',
    'Fair / Windy': 'Windy',
    'Fog / Windy': 'Windy',
    'Haze / Windy': 'Windy',
    'Heavy Rain / Windy': 'Windy',
    'Light Drizzle / Windy': 'Windy',
    'Light Rain / Windy': 'Windy',
    'Mostly Cloudy / Windy': 'Windy',
    'Partly Cloudy / Windy': 'Windy',
    'Rain / Windy': 'Windy',
    'Smoke / Windy': 'Windy',
    'Thunder / Windy': 'Windy',
    'Wintry Mix / Windy': 'Windy',
    'Windy': 'Windy',
    
    # Rainy category
    'Drizzle': 'Rainy',
    'Heavy Rain': 'Rainy',
    'Light Drizzle': 'Rainy',
    'Light Freezing Drizzle': 'Rainy',
    'Light Freezing Rain': 'Rainy',
    'Light Rain': 'Rainy',
    'Light Rain with Thunder': 'Rainy',
    'Rain': 'Rainy',
    'Rainy': 'Rainy',
    
    # Foggy category
    'Drizzle and Fog': 'Foggy',
    'Fog': 'Foggy',
    'Mist': 'Foggy',
    'Patches of Fog': 'Foggy',
    'Shallow Fog': 'Foggy',
    'Haze': 'Foggy',
    'Smoke': 'Foggy',
    'Foggy': 'Foggy',
    'Light Rain / Fog': 'Foggy',
    
    # Clear/Calm category
    'Fair': 'Clear/Calm',
    'Partly Cloudy': 'Clear/Calm',
    
    # Snowy category
    'Snow': 'Snowy',
    'Snow / Windy': 'Snowy',
    'Heavy Snow': 'Snowy',
    'Light Snow': 'Snowy',
    'Light Snow / Windy': 'Snowy',
    'Heavy Snow / Windy': 'Snowy',
    'Wintry Mix': 'Snowy',
    'Blowing Snow': 'Snowy',
    'Snow and Sleet': 'Snowy',
    'Heavy Sleet': 'Snowy',
    'Light Sleet': 'Snowy',
    'Light Snow and Sleet': 'Snowy',
    
    # Stormy category
    'Heavy T-Storm': 'Stormy',
    'T-Storm': 'Stormy',
    'T-Storm / Windy': 'Stormy',
    'Thunder': 'Stormy',
    'Thunder and Hail': 'Stormy',
    'Thunder in the Vicinity': 'Stormy',
    'Small Hail': 'Stormy',
    'Heavy T-Storm / Windy': 'Stormy'
}

# Function to replace subcategories with main categories
def replace_categories(series):
    return series.replace(category_mapping)

# Apply the function to both weather phrase columns
df['wx_phrase_a'] = replace_categories(df['wx_phrase_a'])
df['wx_phrase_d'] = replace_categories(df['wx_phrase_d'])

In [9]:
# Extract the year and create a new column
df['Date (MM/DD/YYYY)_a'] = pd.to_datetime(df['Date (MM/DD/YYYY)_a'])
df['Year'] = df['Date (MM/DD/YYYY)_a'].dt.year

In [45]:
# List of columns to be removed
columns_to_remove = [
    'Flight Number_a', 'Tail Number_a', 'Actual Arrival Time_a', 'Actual Elapsed Time (Minutes)_a',
    'Wheels-on Time_a', 'Taxi-In time (Minutes)_a', 'Delay Carrier (Minutes)_a', 'Delay Weather (Minutes)_a',
    'DateTime_a', 'Unnamed: 0_a', 'expire_time_gmt_a', 'obs_id_a', 'valid_time_gmt_a', 'day_ind_a',
    'wx_icon_a', 'icon_extd_a', 'wc_a', 'wc_d', 'est_time_a', 'est_date_a', 'valid_time_local_a',
    'Unnamed: 0_d_d', 'index_d', 'Carrier Code_d', 'Flight Number_d',
    'Tail Number_d', 'Destination Airport_d', 'Actual departure time_d', 'Scheduled elapsed time (Minutes)_d',
    'Actual elapsed time (Minutes)_d', 'Departure delay (Minutes)_d', 'Wheels-off time_d',
    'Taxi-Out time (Minutes)_d', 'Delay Carrier (Minutes)_d', 'Delay Weather (Minutes)_d', 'Origin Airport_d',
    'DateTime_d', 'Unnamed: 0_w_d', 'expire_time_gmt_d', 'obs_id_d', 'valid_time_gmt_d', 'day_ind_d',
    'wx_icon_d', 'icon_extd_d', 'est_time_d', 'est_date_d', 'valid_time_local_d','wdir_cardinal_d','Date (MM/DD/YYYY)_a',
    'Scheduled Arrival Time_a','wdir_cardinal_a','Carrier Code_a'
]

# Remove the specified columns
df = df.drop(columns=columns_to_remove)

In [101]:
from datetime import datetime, timedelta

# Assuming df is your DataFrame and necessary imports are done
df['Combined_datetime'] = pd.to_datetime(df['Combined_datetime'])
df['date'] = df['Combined_datetime'].dt.date
df['time'] = df['Combined_datetime'].dt.time

# Add an explicit 'index' column
df['index_col'] = df.index

# Reset index before joining
df.reset_index(inplace=True, drop=True)

# Self join on the same date and airport
joined_df = df.merge(df, on=['Origin Airport_a', 'date'], suffixes=('_e', '_l'))

# Calculate the absolute time difference in hours and round to two decimal places
joined_df['time_diff'] = (joined_df['Combined_datetime_e'] - joined_df['Combined_datetime_l']).abs().dt.total_seconds() / 3600
joined_df['time_diff'] = joined_df['time_diff'].round(2)

# Define custom time differences for airports
time_diffs = {
    'ORD': (0.75, 1.3),
    'JFK': (0.75, 2.0),
    'MCO': (0.75, 5)
}

# Function to filter rows based on custom time differences
def custom_time_filter(row):
    lower_bound, upper_bound = time_diffs[row['Origin Airport_a']]
    time_diff = abs(row['Combined_datetime_e'] - row['Combined_datetime_l']).total_seconds() / 3600
    return lower_bound <= time_diff <= upper_bound

# Apply the custom filter
filtered_df = joined_df[joined_df.apply(custom_time_filter, axis=1)]

# Remove duplicate pairs and self-pairs
filtered_df = filtered_df[filtered_df['index_col_e'] < filtered_df['index_col_l']]

# Optionally, drop unnecessary columns
filtered_df.drop(columns=[], inplace=True, errors='ignore')
print(len(filtered_df[filtered_df['Origin Airport_a']=='ORD']))
print(len(filtered_df[filtered_df['Origin Airport_a']=='JFK']))
print(len(filtered_df[filtered_df['Origin Airport_a']=='MCO']))
print(len(filtered_df))

print(filtered_df.head(3))

# a=filtered_df[filtered_df['Date (MM/DD/YYYY)_d_1']=='2023-12-31']
# b= a[a['Origin Airport_a']=='JFK']
# Now merged_df contains merged rows where the earlier flight departure time from the same origin airport is less than 1.5 hours
# print(a)

1437
705
496
2638
    Origin Airport_a  Scheduled Elapsed Time (Minutes)_a_e  \
237              JFK                                  87.0   
412              JFK                                  87.0   
437              JFK                                  89.0   

     Arrival Delay (Minutes)_a_e  \
237                        -23.0   
412                        -21.0   
437                         25.0   

     Delay National Aviation System (Minutes)_a_e  \
237                                           0.0   
412                                           0.0   
437                                          25.0   

     Delay Security (Minutes)_a_e  Delay Late Aircraft Arrival (Minutes)_a_e  \
237                           0.0                                        0.0   
412                           0.0                                        0.0   
437                           0.0                                        0.0   

     temp_a_e wx_phrase_a_e  dewPt_a_e  heat_index_a_e

In [ ]:
# List of columns to be removed
columns_to_remove = [
    'Flight Number_a', 'Tail Number_a', 'Actual Arrival Time_a', 'Actual Elapsed Time (Minutes)_a',
    'Wheels-on Time_a', 'Taxi-In time (Minutes)_a', 'Delay Carrier (Minutes)_a', 'Delay Weather (Minutes)_a',
    'DateTime_a', 'Unnamed: 0_a', 'expire_time_gmt_a', 'obs_id_a', 'valid_time_gmt_a', 'day_ind_a',
    'wx_icon_a', 'icon_extd_a', 'wc_a', 'wc_d', 'est_time_a', 'est_date_a', 'valid_time_local_a',
    'Unnamed: 0_d_d', 'index_d', 'Carrier Code_d', 'Date (MM/DD/YYYY)_d', 'Flight Number_d',
    'Tail Number_d', 'Destination Airport_d', 'Actual departure time_d', 'Scheduled elapsed time (Minutes)_d',
    'Actual elapsed time (Minutes)_d', 'Departure delay (Minutes)_d', 'Wheels-off time_d',
    'Taxi-Out time (Minutes)_d', 'Delay Carrier (Minutes)_d', 'Delay Weather (Minutes)_d', 'Origin Airport_d',
    'DateTime_d', 'Unnamed: 0_w_d', 'expire_time_gmt_d', 'obs_id_d', 'valid_time_gmt_d', 'day_ind_d',
    'wx_icon_d', 'icon_extd_d', 'est_time_d', 'est_date_d', 'valid_time_local_d','wdir_cardinal_d','Date (MM/DD/YYYY)_a',
    'Scheduled Arrival Time_a','wdir_cardinal_a','Carrier Code_a','Scheduled departure time_d'
]

# Remove the specified columns
df = df.drop(columns=columns_to_remove)

In [15]:
print(df.iloc[0])

22:10


In [53]:
# List of columns to check for unique values
columns_to_check = [
    'Origin Airport_a', 'wx_phrase_a', 'uv_desc_a', 'clds_a',
    'wx_phrase_d', 'uv_desc_d', 'clds_d', 'Main_Carrier',
    'Week_Category', 'A_Time_of_Day'
]

# Print all unique values for each of the listed columns
for column in columns_to_check:
    print(f'Unique values in {column}: {df[column].unique()}')

Unique values in Origin Airport_a: ['ORD' 'JFK' 'MCO']
Unique values in wx_phrase_a: ['Cloudy' 'Snowy' 'Stormy' 'Clear/Calm' 'Rainy' 'Foggy' 'Windy']
Unique values in uv_desc_a: ['Low' 'Moderate' 'High']
Unique values in clds_a: ['OVC' 'BKN' 'SCT' 'FEW' 'CLR']
Unique values in wx_phrase_d: ['Cloudy' 'Windy' 'Clear/Calm' 'Snowy' 'Foggy' 'Rainy' 'Stormy']
Unique values in uv_desc_d: ['Low' 'Moderate' 'High']
Unique values in clds_d: ['OVC' 'BKN' 'SCT' 'FEW' 'CLR']
Unique values in Main_Carrier: ['AA' 'B6' 'DL' 'UA' 'WN']
Unique values in Week_Category: ['Midweek' 'Weekend' 'StartOfWeek']
Unique values in A_Time_of_Day: ['Night' 'Morning' 'Morning Peak' 'Afternoon' 'Evening Peak' 'Evening']


In [54]:
print(df.iloc[0])

Origin Airport_a                                  ORD
Scheduled Elapsed Time (Minutes)_a              104.0
Arrival Delay (Minutes)_a                         0.0
Delay National Aviation System (Minutes)_a        0.0
Delay Security (Minutes)_a                        0.0
Delay Late Aircraft Arrival (Minutes)_a           0.0
temp_a                                           48.0
wx_phrase_a                                    Cloudy
dewPt_a                                          42.0
heat_index_a                                     48.0
rh_a                                             80.0
pressure_a                                      29.05
vis_a                                            10.0
wspd_a                                           12.0
precip_total_a                                    0.0
precip_hrly_a                                     0.0
snow_hrly_a                                       0.0
uv_desc_a                                         Low
feels_like_a                

In [110]:
filtered_df.to_csv('cleaned_adw_df_22.csv', index=False)

In [102]:
filtered_df.head(1)

Origin Airport_a  Scheduled Elapsed Time (Minutes)_a_e  \
237              JFK                                  87.0   

     Arrival Delay (Minutes)_a_e  \
237                        -23.0   

     Delay National Aviation System (Minutes)_a_e  \
237                                           0.0   

     Delay Security (Minutes)_a_e  Delay Late Aircraft Arrival (Minutes)_a_e  \
237                           0.0                                        0.0   

     temp_a_e wx_phrase_a_e  dewPt_a_e  heat_index_a_e  rh_a_e  pressure_a_e  \
237      14.0        Cloudy        2.0            14.0    59.0         30.09   

     vis_a_e  wspd_a_e  precip_total_a_e  precip_hrly_a_e  snow_hrly_a_e  \
237     10.0       6.0               0.0              0.0            0.0   

    uv_desc_a_e  feels_like_a_e  uv_index_a_e clds_a_e Date (MM/DD/YYYY)_d_e  \
237         Low             5.0           0.0      BKN            01/12/2019   

    Scheduled departure time_d_e  \
237                     16:45:00   

     Delay National Aviation System (Minutes)_d_e  \
237                                             0   

     Delay Security (Minutes)_d_e  Delay Late Aircraft Arrival (Minutes)_d_e  \
237                             0                                          0   

     temp_d_e wx_phrase_d_e  dewPt_d_e  heat_index_d_e  rh_d_e  pressure_d_e  \
237      33.0        Cloudy        8.0            33.0    35.0         30.49   

     vis_d_e  wspd_d_e  precip_total_d_e  precip_hrly_d_e  snow_hrly_d_e  \
237     10.0       6.0               0.0              0.0            0.0   

    uv_desc_d_e  feels_like_d_e  uv_index_d_e clds_d_e Main_Carrier_e  \
237         Low            28.0           0.0      BKN             DL   

    Week_Category_e A_Time_of_Day_e  Year_e Combined_datetime_e  \
237         Weekend         Evening    2019 2019-01-12 16:45:00   

    TimeDifference_e        date    time_e  index_col_e  \
237  0 days 07:15:00  2019-01-12  16:45:00           50   

     Scheduled Elapsed Time (Minutes)_a_l  Arrival Delay (Minutes)_a_l  \
237                                  78.0                        -14.0   

     Delay National Aviation System (Minutes)_a_l  \
237                                           0.0   

     Delay Security (Minutes)_a_l  Delay Late Aircraft Arrival (Minutes)_a_l  \
237                           0.0                                        0.0   

     temp_a_l wx_phrase_a_l  dewPt_a_l  heat_index_a_l  rh_a_l  pressure_a_l  \
237      13.0        Cloudy        0.0            13.0    56.0          30.1   

     vis_a_l  wspd_a_l  precip_total_a_l  precip_hrly_a_l  snow_hrly_a_l  \
237      9.0       0.0               0.0              0.0            0.0   

    uv_desc_a_l  feels_like_a_l  uv_index_a_l clds_a_l Date (MM/DD/YYYY)_d_l  \
237         Low            13.0           0.0      BKN            01/12/2019   

    Scheduled departure time_d_l  \
237                     17:38:00   

     Delay National Aviation System (Minutes)_d_l  \
237                                             0   

     Delay Security (Minutes)_d_l  Delay Late Aircraft Arrival (Minutes)_d_l  \
237                             0                                          0   

     temp_d_l wx_phrase_d_l  dewPt_d_l  heat_index_d_l  rh_d_l  pressure_d_l  \
237      31.0        Cloudy        6.0            31.0    35.0         30.49   

     vis_d_l  wspd_d_l  precip_total_d_l  precip_hrly_d_l  snow_hrly_d_l  \
237     10.0       7.0               0.0              0.0            0.0   

    uv_desc_d_l  feels_like_d_l  uv_index_d_l clds_d_l Main_Carrier_l  \
237         Low            24.0           0.0      BKN             B6   

    Week_Category_l A_Time_of_Day_l  Year_l Combined_datetime_l  \
237         Weekend         Evening    2019 2019-01-12 17:38:00   

    TimeDifference_l    time_l  index_col_l  time_diff  
237  0 days 00:53:00  17:38:00           51       0.88